# RAS Commander: Plan Key Operations

This notebook demonstrates how to perform key operations on HEC-RAS plan files using the RAS Commander library. Plan files in HEC-RAS (`.p*` files) control the simulation settings and parameters, making them essential for hydraulic modeling workflows.

## Operations Covered

1. **Project Initialization**: Set up a HEC-RAS project for automation
2. **Plan Values**: Retrieve specific values from plan files
3. **Run Flags**: Configure which components (geometry preprocessor, unsteady flow, etc.) will run
4. **Plan Intervals**: Set computation and output time intervals
5. **Plan Descriptions**: Read and update plan descriptions
6. **Simulation Dates**: Modify simulation start and end dates

These operations allow you to programmatically control and customize HEC-RAS simulations without opening the GUI, which is especially useful for batch processing, sensitivity analysis, and model calibration.

Let's begin by importing the necessary libraries and setting up our environment.

## Package Installation and Environment Setup
Uncomment and run package installation commands if needed

In [1]:
# Install ras-commander from pip (uncomment to install if needed)
#!pip install ras-commander
# This installs ras-commander and all dependencies

In [2]:
# Import all required modules

# Import all ras-commander modules
from ras_commander import *

# Import the required libraries for this notebook
import numpy as np
import pandas as pd
from IPython import display
from pathlib import Path
import matplotlib.pyplot as plt
import psutil  # For getting system CPU info
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import subprocess
import sys
import os
import shutil
from datetime import datetime, timedelta

In [ ]:
# Extract the Bald Eagle Creek example project
# The extract_project method downloads the project from GitHub if not already present,
# and extracts it to the example_projects folder
bald_eagle_path = RasExamples.extract_project("Balde Eagle Creek")
print(f"Extracted project to: {bald_eagle_path}")  


# Verify the path exists
print(f"Bald Eagle Creek project exists: {bald_eagle_path.exists()}")

## Understanding Plan Files in HEC-RAS

Before we dive into the operations, let's understand what HEC-RAS plan files are and why they're important:

### What is a Plan File?

A HEC-RAS plan file (`.p*`) is a configuration file that defines how a hydraulic simulation will run. It links together:

1. **Geometry**: River channel and floodplain physical characteristics (`.g*` files)
2. **Flow Data**: Inflow conditions, either steady (`.f*`) or unsteady (`.u*`)
3. **Simulation Parameters**: Time steps, computational methods, and output settings

### Key Components of Plan Files

Plan files contain many parameters that control simulation behavior:

- **Simulation Type**: Steady, unsteady, sediment transport, water quality
- **Computation Intervals**: Time steps for calculations
- **Output Intervals**: How frequently results are saved
- **Run Flags**: Which modules to execute (preprocessor, postprocessor, etc.)
- **Simulation Period**: Start and end dates for unsteady simulations
- **Computation Methods**: Numerical schemes and solver settings
- **Resource Allocation**: Number of CPU cores to use

### Why Automate Plan Operations?

Automating plan operations with RAS Commander allows you to:

1. **Batch Processing**: Run multiple scenarios with different parameters
2. **Sensitivity Analysis**: Systematically vary parameters to assess their impact
3. **Calibration**: Adjust parameters to match observed data
4. **Consistency**: Ensure standardized settings across multiple models
5. **Documentation**: Programmatically track simulation configurations

Now, let's download and extract an example project to work with.

In [ ]:
# Create a RasExamples instance
ras_examples = RasExamples()

# Extract the Bald Eagle Creek example project
extracted_paths = ras_examples.extract_project(["Balde Eagle Creek"])
print(f"Extracted project to: {extracted_paths}")

# Verify the path exists
print(f"Bald Eagle Creek project exists: {bald_eagle_path.exists()}")

## Step 1: Project Initialization

The first step in any RAS Commander workflow is initializing the HEC-RAS project. This connects the Python environment to the HEC-RAS project files.

The `init_ras_project()` function does the following:

1. Locates the main project file (`.prj`)
2. Reads all associated files (plans, geometries, flows)
3. Creates dataframes containing project components
4. Sets up the connection to the HEC-RAS executable

Let's initialize our project:

In [ ]:
# Initialize the project (using the default global ras object)
init_ras_project(bald_eagle_path, "6.6")
print(f"Initialized project: {ras.project_name}")

# Display basic project information
print("\nProject Overview:")
print(f"Project Folder: {ras.project_folder}")
print(f"Project File: {ras.prj_file}")
print(f"Number of Plan Files: {len(ras.plan_df)}")
print(f"Number of Geometry Files: {len(ras.geom_df)}")
print(f"Number of Flow Files: {len(ras.flow_df)}")
print(f"Number of Unsteady Files: {len(ras.unsteady_df)}")

Let's also take a look at the plan files in this project:

In [ ]:
# Display the plan files
print("Plan Files in Project:")
display.display(ras.plan_df[['plan_number', 'Plan Title', 'Short Identifier', 'Geom File']])

# Get the first plan number for our examples
plan_number = ras.plan_df['plan_number'].iloc[0]
print(f"\nWe'll work with Plan: {plan_number}")

## Step 2: Retrieving Plan Values

The `RasPlan.get_plan_value()` method allows you to retrieve specific values from a plan file. This is useful for checking current settings before making changes or for extracting information for analysis.

### Key Parameters

- `plan_number_or_path` (str or Path): The plan number or full path to the plan file
- `key` (str): The specific parameter to retrieve (e.g., "Computation Interval")
- `rasect` (RasPrj, optional): The RAS project object (defaults to global `ras`)

### Common Keys

- `Computation Interval`: Time step for calculations (e.g., "5SEC", "1MIN")
- `Short Identifier`: Brief name/ID for the plan
- `Simulation Date`: Start and end dates for simulation
- `UNET D1 Cores`: Number of processor cores to use
- `Plan Title`: Full title of the plan
- `Geom File`: Associated geometry file
- `Flow File`: Associated flow file (for steady flow)
- `Unsteady File`: Associated unsteady flow file
- `Friction Slope Method`: Method for calculating friction slopes
- `Run HTab`: Whether to run the geometry preprocessor
- `UNET Use Existing IB Tables`: Whether to use existing internal boundary tables

Let's retrieve some key values from our plan:

In [ ]:
# Define keys to check
keys_to_check = [
    'Short Identifier', 
    'Plan Title',
    'Computation Interval', 
    'Simulation Date', 
    'UNET D1 Cores',
    'Geom File',
    'Friction Slope Method'
]

# Retrieve and display the initial values
print("Initial Plan Values:")
initial_values = {}
for key in keys_to_check:
    value = RasPlan.get_plan_value(plan_number, key, rasect=ras)
    initial_values[key] = value
    print(f"  {key}: {value}")

# DEV NOTE: NEED TO REVIEW # OF CORES LOGIC TO ENSURE IT IS UPDATED AND APPLIES TO 1D/2D and PIPES

## Step 3: Updating Run Flags

Run flags in HEC-RAS control which components of the simulation are executed. The `RasPlan.update_run_flags()` method allows you to modify these flags programmatically.

### Key Parameters

- `plan_number_or_path` (str or Path): The plan number or full path to the plan file
- `geometry_preprocessor` (bool, optional): Whether to run the geometry preprocessor
- `unsteady_flow_simulation` (bool, optional): Whether to run the unsteady flow simulation
- `run_sediment` (bool, optional): Whether to run sediment transport calculations
- `post_processor` (bool, optional): Whether to run the post-processor
- `floodplain_mapping` (bool, optional): Whether to run floodplain mapping
- `rasect` (RasPrj, optional): The RAS project object

### Common Run Flags

1. **Geometry Preprocessor**: Computes hydraulic tables from geometry data
   - `True`: Recompute tables (useful after geometry changes)
   - `False`: Use existing tables (faster but may be outdated)

2. **Unsteady Flow Simulation**: The main hydraulic calculations
   - `True`: Run unsteady flow calculations
   - `False`: Skip unsteady flow calculations

3. **Sediment Transport**: Simulates erosion and deposition
   - `True`: Calculate sediment transport
   - `False`: Skip sediment transport

4. **Post-Processor**: Calculates additional variables from results
   - `True`: Run post-processing (recommended)
   - `False`: Skip post-processing (faster but fewer outputs)

5. **Floodplain Mapping**: Generates inundation maps
   - `True`: Generate maps (requires terrain data)
   - `False`: Skip mapping (faster)

Let's update the run flags for our plan:

In [ ]:
# Update run flags for the plan
print(f"Updating run flags for plan {plan_number}...")
RasPlan.update_run_flags(
    plan_number,
    geometry_preprocessor=True,      # Force recalculation of hydraulic tables
    unsteady_flow_simulation=True,   # Run the main hydraulic calculations
    run_sediment=False,              # Skip sediment transport calculations
    post_processor=True,             # Run post-processing for additional outputs
    floodplain_mapping=False,        # Skip floodplain mapping
    rasect=ras
)
print("Run flags updated successfully")

## Step 4: Updating Plan Intervals

Time intervals in HEC-RAS control the temporal resolution of simulations and outputs. The `RasPlan.update_plan_intervals()` method allows you to modify these intervals.

### Key Parameters

- `plan_number_or_path` (str or Path): The plan number or full path to the plan file
- `computation_interval` (str, optional): Time step for calculations
- `output_interval` (str, optional): Time step for saving detailed results
- `instantaneous_interval` (str, optional): Time step for peak value calculations
- `mapping_interval` (str, optional): Time step for map outputs
- `rasect` (RasPrj, optional): The RAS project object

### Valid Interval Values

Time intervals must be specified in HEC-RAS format:
- Seconds: `1SEC`, `2SEC`, `3SEC`, `4SEC`, `5SEC`, `6SEC`, `10SEC`, `15SEC`, `20SEC`, `30SEC`
- Minutes: `1MIN`, `2MIN`, `3MIN`, `4MIN`, `5MIN`, `6MIN`, `10MIN`, `15MIN`, `20MIN`, `30MIN`
- Hours: `1HOUR`, `2HOUR`, `3HOUR`, `4HOUR`, `6HOUR`, `8HOUR`, `12HOUR`
- Days: `1DAY`

### Interval Types

1. **Computation Interval**: Time step used for hydraulic calculations
   - Smaller intervals: More accurate but slower
   - Larger intervals: Faster but may introduce numerical errors
   - Rule of thumb: Should be small enough to capture flow changes

2. **Output Interval**: How frequently detailed results are saved
   - Smaller intervals: More detailed results but larger files
   - Larger intervals: Smaller files but less temporal resolution
   - Usually larger than computation interval

3. **Instantaneous Interval**: Time step for peak value calculations
   - Affects when max/min values are checked
   - Usually equal to output interval

4. **Mapping Interval**: How frequently map data is saved
   - Affects animation smoothness and file size
   - Usually larger than output interval

Let's update the intervals for our plan:

In [ ]:
# Update plan intervals
print(f"Updating intervals for plan {plan_number}...")
RasPlan.update_plan_intervals(
    plan_number,
    computation_interval="5SEC",    # 5-second time step for calculations
    output_interval="1MIN",         # Save detailed results every minute
    instantaneous_interval="5MIN",  # Check for max/min values every 5 minutes
    mapping_interval="15MIN",       # Save map data every 15 minutes
    rasect=ras
)
print("Plan intervals updated successfully")

## Step 5: Managing Plan Descriptions

Plan descriptions provide documentation for simulation configurations. The RAS Commander library offers methods to read and update these descriptions.

### Reading Descriptions

The `RasPlan.read_plan_description()` method retrieves the current description from a plan file.

#### Parameters
- `plan_number_or_path` (str or Path): The plan number or full path to the plan file
- `rasect` (RasPrj, optional): The RAS project object

### Updating Descriptions

The `RasPlan.update_plan_description()` method sets a new description for a plan file.

#### Parameters
- `plan_number_or_path` (str or Path): The plan number or full path to the plan file
- `description` (str): The new description text
- `rasect` (RasPrj, optional): The RAS project object

### Best Practices for Plan Descriptions

Effective plan descriptions should include:
1. Purpose of the simulation
2. Key parameters and settings
3. Date of creation or modification
4. Author or organization
5. Any special considerations or notes

Let's read the current description and then update it:

In [ ]:
# Read the current plan description
current_description = RasPlan.read_plan_description(plan_number, rasect=ras)
print(f"Current plan description:\n{current_description}")

# Create a new description with detailed information
new_description = f"""Modified Plan for RAS Commander Testing
Date: {datetime.now().strftime('%Y-%m-%d')}
Purpose: Demonstrating RAS Commander plan operations
Settings:
- Computation Interval: 5SEC
- Output Interval: 1MIN
- Mapping Interval: 15MIN
- Geometry Preprocessor: Enabled
- Post-Processor: Enabled
Notes: This plan was automatically modified using ras-commander."""

# Update the plan description
print("\nUpdating plan description...")
RasPlan.update_plan_description(plan_number, new_description, rasect=ras)
print("Plan description updated successfully")

# Verify the updated description
updated_description = RasPlan.read_plan_description(plan_number, rasect=ras)
print(f"\nUpdated plan description:\n{updated_description}")

## Step 6: Updating Simulation Dates

For unsteady flow simulations, the simulation period defines the time window for the analysis. The `RasPlan.update_simulation_date()` method allows you to modify this period.

### Key Parameters

- `plan_number_or_path` (str or Path): The plan number or full path to the plan file
- `start_date` (datetime): The start date and time for the simulation
- `end_date` (datetime): The end date and time for the simulation
- `rasect` (RasPrj, optional): The RAS project object

### Considerations for Simulation Dates

1. **Hydrograph Coverage**: The simulation period should fully encompass your hydrographs
2. **Warm-Up Period**: Include time before the main event for model stabilization
3. **Cool-Down Period**: Include time after the main event for complete drainage
4. **Computational Efficiency**: Avoid unnecessarily long periods to reduce runtime
5. **Consistency**: Ensure dates match available boundary condition data

Let's update the simulation dates for our plan:

In [ ]:
# Get the current simulation date
current_sim_date = RasPlan.get_plan_value(plan_number, "Simulation Date", rasect=ras)
print(f"Current simulation date: {current_sim_date}")

# Parse the current simulation date string
current_dates = current_sim_date.split(",")
current_start = datetime.strptime(f"{current_dates[0]},{current_dates[1]}", "%d%b%Y,%H%M")
current_end = datetime.strptime(f"{current_dates[2]},{current_dates[3]}", "%d%b%Y,%H%M")

# Define new simulation period - adjust by 1 hour from current dates
start_date = current_start + timedelta(hours=1)  # Current start + 1 hour
end_date = current_end - timedelta(hours=1)      # Current end - 1 hour

# Update the simulation date
print(f"\nUpdating simulation period to: {start_date.strftime('%d%b%Y,%H%M')} - {end_date.strftime('%d%b%Y,%H%M')}")
RasPlan.update_simulation_date(plan_number, start_date, end_date, rasect=ras)
print("Simulation dates updated successfully")

# Verify the updated simulation date
updated_sim_date = RasPlan.get_plan_value(plan_number, "Simulation Date", rasect=ras)
print(f"\nUpdated simulation date: {updated_sim_date}")

## Step 7: Verifying Updated Plan Values

After making multiple changes to a plan, it's a good practice to verify that all updates were applied correctly. Let's check the updated values:

In [ ]:
# Retrieve and display the updated values
print("Updated Plan Values:")
updated_values = {}
for key in keys_to_check:
    value = RasPlan.get_plan_value(plan_number, key, rasect=ras)
    updated_values[key] = value
    print(f"  {key}: {value}")

# Create a comparison dataframe
comparison_df = pd.DataFrame({
    'Parameter': keys_to_check,
    'Initial Value': [initial_values.get(k, 'N/A') for k in keys_to_check],
    'Updated Value': [updated_values.get(k, 'N/A') for k in keys_to_check]
})

print("\nChanges Summary:")
display.display(comparison_df)

## Step 8: Computing the Plan (Optional)

After making changes to a plan, you might want to run the simulation to see the effects. The `RasCmdr.compute_plan()` method executes a HEC-RAS simulation with the specified plan.

### Key Parameters

- `plan_number` (str): The plan number to execute
- `dest_folder` (str, Path, optional): Destination folder for computation
- `rasect` (RasPrj, optional): The RAS project object
- `clear_geompre` (bool, optional): Whether to clear geometry preprocessor files
- `num_cores` (int, optional): Number of processor cores to use
- `overwrite_dest` (bool, optional): Whether to overwrite the destination folder

If you want to run the simulation, you can uncomment the code below:

# DEV NOTE THIS SHOULD RUN!

In [13]:
# Uncomment to run the simulation with the updated plan

# # Define a destination folder for the computation
# dest_folder = script_dir / "compute_results"
# print(f"Computing plan {plan_number}...")
# print(f"Results will be saved to: {dest_folder}")

# # Execute the plan
# success = RasCmdr.compute_plan(
#     plan_number,
#     dest_folder=dest_folder,
#     clear_geompre=True,    # Clear preprocessor files to ensure clean results
#     num_cores=2,           # Use 2 processor cores
#     overwrite_dest=True,   # Overwrite existing destination folder
#     rasect=ras
# )

# if success:
#     print(f"Plan {plan_number} computed successfully")
#     # Check for results file
#     results_path = RasPlan.get_results_path(plan_number, rasect=ras)
#     if results_path:
#         print(f"Results saved to: {results_path}")
# else:
#     print(f"Failed to compute plan {plan_number}")

## Summary of Plan Key Operations

In this notebook, we've covered the essential operations for manipulating HEC-RAS plan files programmatically using RAS Commander:

1. **Project Initialization**: We initialized a HEC-RAS project using `init_ras_project()`
2. **Plan Values**: We retrieved plan values with `RasPlan.get_plan_value()`
3. **Run Flags**: We updated simulation components with `RasPlan.update_run_flags()`
4. **Plan Intervals**: We modified time steps with `RasPlan.update_plan_intervals()`
5. **Plan Descriptions**: We managed documentation with `RasPlan.read_plan_description()` and `RasPlan.update_plan_description()`
6. **Simulation Dates**: We changed the analysis period with `RasPlan.update_simulation_date()`
7. **Verification**: We verified our changes by comparing initial and updated values

### Key Classes and Functions Used

- `RasPlan`: The main class for plan operations
  - `get_plan_value()`: Retrieve specific values from plan files
  - `update_run_flags()`: Configure which components will run
  - `update_plan_intervals()`: Set computation and output time intervals
  - `read_plan_description()`: Get the current plan description
  - `update_plan_description()`: Set a new plan description
  - `update_simulation_date()`: Modify the simulation period
  - `get_results_path()`: Get the path to results files

- `RasCmdr`: The class for executing HEC-RAS simulations
  - `compute_plan()`: Run a single plan simulation

### Best Practices for Plan Operations

1. **Verify Before Updating**: Always check current values before making changes
2. **Document Changes**: Use descriptive plan descriptions to track modifications
3. **Maintain Consistency**: Ensure flow data matches simulation dates
4. **Use Appropriate Intervals**: Balance accuracy and computational efficiency
5. **Backup Original Files**: Use destination folders when running simulations
6. **Verify After Updates**: Confirm that all changes were applied correctly

### Next Steps

With these plan operations, you can now:

1. **Create Batch Workflows**: Process multiple scenarios with different parameters
2. **Perform Sensitivity Analysis**: Systematically vary parameters to assess their impact
3. **Automate Calibration**: Adjust parameters to match observed data
4. **Build Model Ensembles**: Run multiple configurations for uncertainty analysis
5. **Integrate with Other Tools**: Connect HEC-RAS to broader modeling frameworks

These operations form the foundation for advanced HEC-RAS automation using the RAS Commander library.